In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-payment-fraud-detection-dataset/Fraud Detection Dataset.csv


In [2]:
import matplotlib.pyplot as plt  # data visualization
import seaborn as sns  # enhanced data visualization

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv("/kaggle/input/online-payment-fraud-detection-dataset/Fraud Detection Dataset.csv",low_memory=False)


In [4]:
data

,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,User's Transaction History,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag
0,51595306,9822,163.08,2023-01-02 07:47:54,4044,ACH Transfer,KOR,Charity,GPS Device,42.23.223.120,...,26,2.71,United Kingdom,NOK,Consultation Fee,343,cox.co.uk,3,Bluetooth Authentication,0
1,85052974,4698,430.74,2021-09-12 15:15:41,4576,2Checkout,VNM,Cashback,Medical Device,39.52.212.120,...,60,3.95,Mexico,EGP,Cashback Reward,688,gmail.com,13,NFC Tag,1
2,23954324,8666,415.74,2023-01-12 17:25:58,4629,Google Wallet,MEX,Reward,Vehicle Infotainment System,243.180.236.29,...,81,3.81,Qatar,MXN,Acquisition,371,rocketmail.com,7,Token,1
3,44108303,9012,565.89,2021-02-27 11:31:00,3322,Check,SGP,Purchase,Kiosk,212.186.227.14,...,18,2.67,Spain,CLP,Loan Repayment,687,roadrunner.co.uk,15,Time-Based OTP,1
4,66622683,5185,955.49,2022-09-24 04:06:38,7609,Worldpay,HKG,Acquisition,Smart Mirror,166.113.10.199,...,98,3.19,Israel,RUB,Dividend Reinvestment,605,protonmail.co.uk,17,Password,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,61037029,7480,448.99,2021-10-20 15:56:32,3346,Discover,SGP,Scholarship,Server,255.134.160.201,...,34,2.78,Russia,CHF,Invoice Payment,679,aim.com,14,Retina Scan,0
5999996,56515851,5636,841.39,2021-06-14 02:10:00,8415,Alipay,ZAF,Loan,Digital Camera,48.190.84.14,...,80,2.60,Malaysia,HUF,Membership,706,cox.net,10,Social Media Login,1
5999997,66863972,5554,197.28,2021-11-06 22:33:19,4231,Afterpay,CAN,Service Charge,Barcode Scanner,7.21.196.39,...,12,1.35,Egypt,HKD,Admission,310,live.co.uk,14,Mobile App Notification,0
5999998,13449701,1275,358.33,2022-03-13 15:02:35,9614,JCB,UK,Fine,Robot,211.202.242.100,...,57,1.29,China,AED,Expense Reimbursement,460,rediffmail.com,16,Authentication App,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000000 entries, 0 to 5999999
Data columns (total 32 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   Transaction ID                     int64  
 1   User ID                            int64  
 2   Transaction Amount                 float64
 3   Transaction Date and Time          object 
 4   Merchant ID                        int64  
 5   Payment Method                     object 
 6   Country Code                       object 
 7   Transaction Type                   object 
 8   Device Type                        object 
 9   IP Address                         object 
 10  Browser Type                       object 
 11  Operating System                   object 
 12  Merchant Category                  object 
 13  User Age                           int64  
 14  User Occupation                    object 
 15  User Income                        float64
 16  User Gender       

## Reducing Memory Usage

In [6]:
#Reduce Memory Usage
def reduce_memory_usage(df):
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    
    return df

In [7]:
reduce_memory_usage(data)

,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,User's Transaction History,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag
0,51595306,9822,163.125,2023-01-02 07:47:54,4044,ACH Transfer,KOR,Charity,GPS Device,42.23.223.120,...,26,2.710938,United Kingdom,NOK,Consultation Fee,343,cox.co.uk,3,Bluetooth Authentication,0
1,85052974,4698,430.750,2021-09-12 15:15:41,4576,2Checkout,VNM,Cashback,Medical Device,39.52.212.120,...,60,3.949219,Mexico,EGP,Cashback Reward,688,gmail.com,13,NFC Tag,1
2,23954324,8666,415.750,2023-01-12 17:25:58,4629,Google Wallet,MEX,Reward,Vehicle Infotainment System,243.180.236.29,...,81,3.810547,Qatar,MXN,Acquisition,371,rocketmail.com,7,Token,1
3,44108303,9012,566.000,2021-02-27 11:31:00,3322,Check,SGP,Purchase,Kiosk,212.186.227.14,...,18,2.669922,Spain,CLP,Loan Repayment,687,roadrunner.co.uk,15,Time-Based OTP,1
4,66622683,5185,955.500,2022-09-24 04:06:38,7609,Worldpay,HKG,Acquisition,Smart Mirror,166.113.10.199,...,98,3.189453,Israel,RUB,Dividend Reinvestment,605,protonmail.co.uk,17,Password,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,61037029,7480,449.000,2021-10-20 15:56:32,3346,Discover,SGP,Scholarship,Server,255.134.160.201,...,34,2.779297,Russia,CHF,Invoice Payment,679,aim.com,14,Retina Scan,0
5999996,56515851,5636,841.500,2021-06-14 02:10:00,8415,Alipay,ZAF,Loan,Digital Camera,48.190.84.14,...,80,2.599609,Malaysia,HUF,Membership,706,cox.net,10,Social Media Login,1
5999997,66863972,5554,197.250,2021-11-06 22:33:19,4231,Afterpay,CAN,Service Charge,Barcode Scanner,7.21.196.39,...,12,1.349609,Egypt,HKD,Admission,310,live.co.uk,14,Mobile App Notification,0
5999998,13449701,1275,358.250,2022-03-13 15:02:35,9614,JCB,UK,Fine,Robot,211.202.242.100,...,57,1.290039,China,AED,Expense Reimbursement,460,rediffmail.com,16,Authentication App,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000000 entries, 0 to 5999999
Data columns (total 32 columns):
 #   Column                             Dtype   
---  ------                             -----   
 0   Transaction ID                     int32   
 1   User ID                            int16   
 2   Transaction Amount                 float16 
 3   Transaction Date and Time          category
 4   Merchant ID                        int16   
 5   Payment Method                     category
 6   Country Code                       category
 7   Transaction Type                   category
 8   Device Type                        category
 9   IP Address                         category
 10  Browser Type                       category
 11  Operating System                   category
 12  Merchant Category                  category
 13  User Age                           int8    
 14  User Occupation                    category
 15  User Income                        float32 
 16  

In [9]:
data

,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,User's Transaction History,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag
0,51595306,9822,163.125,2023-01-02 07:47:54,4044,ACH Transfer,KOR,Charity,GPS Device,42.23.223.120,...,26,2.710938,United Kingdom,NOK,Consultation Fee,343,cox.co.uk,3,Bluetooth Authentication,0
1,85052974,4698,430.750,2021-09-12 15:15:41,4576,2Checkout,VNM,Cashback,Medical Device,39.52.212.120,...,60,3.949219,Mexico,EGP,Cashback Reward,688,gmail.com,13,NFC Tag,1
2,23954324,8666,415.750,2023-01-12 17:25:58,4629,Google Wallet,MEX,Reward,Vehicle Infotainment System,243.180.236.29,...,81,3.810547,Qatar,MXN,Acquisition,371,rocketmail.com,7,Token,1
3,44108303,9012,566.000,2021-02-27 11:31:00,3322,Check,SGP,Purchase,Kiosk,212.186.227.14,...,18,2.669922,Spain,CLP,Loan Repayment,687,roadrunner.co.uk,15,Time-Based OTP,1
4,66622683,5185,955.500,2022-09-24 04:06:38,7609,Worldpay,HKG,Acquisition,Smart Mirror,166.113.10.199,...,98,3.189453,Israel,RUB,Dividend Reinvestment,605,protonmail.co.uk,17,Password,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,61037029,7480,449.000,2021-10-20 15:56:32,3346,Discover,SGP,Scholarship,Server,255.134.160.201,...,34,2.779297,Russia,CHF,Invoice Payment,679,aim.com,14,Retina Scan,0
5999996,56515851,5636,841.500,2021-06-14 02:10:00,8415,Alipay,ZAF,Loan,Digital Camera,48.190.84.14,...,80,2.599609,Malaysia,HUF,Membership,706,cox.net,10,Social Media Login,1
5999997,66863972,5554,197.250,2021-11-06 22:33:19,4231,Afterpay,CAN,Service Charge,Barcode Scanner,7.21.196.39,...,12,1.349609,Egypt,HKD,Admission,310,live.co.uk,14,Mobile App Notification,0
5999998,13449701,1275,358.250,2022-03-13 15:02:35,9614,JCB,UK,Fine,Robot,211.202.242.100,...,57,1.290039,China,AED,Expense Reimbursement,460,rediffmail.com,16,Authentication App,0


# Data Cleaning

## Checking for Missing Values in the Dataset

In [10]:
# Check for missing values in the entire DataFrame
print(data.isnull().sum())

Transaction ID                       0
User ID                              0
Transaction Amount                   0
Transaction Date and Time            0
Merchant ID                          0
Payment Method                       0
Country Code                         0
Transaction Type                     0
Device Type                          0
IP Address                           0
Browser Type                         0
Operating System                     0
Merchant Category                    0
User Age                             0
User Occupation                      0
User Income                          0
User Gender                          0
User Account Status                  0
Transaction Status                   0
Location Distance                    0
Time Taken for Transaction           0
Transaction Time of Day              0
User's Transaction History           0
Merchant's Reputation Score          0
User's Device Location               0
Transaction Currency     

## Checking for Duplicate Transactions in the Dataset

In [11]:
duplicate_rows = data.duplicated()
duplicate_rows_df = data[duplicate_rows]
duplicate_rows_df

,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,User's Transaction History,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag


## Checking for Duplicate Entries in Transaction ID

### Background Story
##### In a typical and well-designed transaction system, each transaction is usually assigned a unique identifier known as the transaction ID. This ID is used to distinguish one transaction from another and is crucial for tracking and managing transactions within a system.

##### Therefore, in a properly functioning transaction system, it is highly improbable for two transactions to have the same transaction ID. The uniqueness of transaction IDs is essential to ensure accurate tracking, auditing, and management of transactions. If two transactions were to have the same ID, it could lead to confusion, incorrect processing, and potential data integrity issues.

### Observation
##### It is observed from the investigation below that:
#####  
+ There are 5805013 unique Transaction ID out of the 6000000 rows in the dataset. This means we have transactions with the same ID.
+ On further investigation, there are 385737 rows which share Transaction ID. 

In [12]:
data['Transaction ID'].nunique()

5805013

In [13]:
data = data.sort_values(by='Transaction ID', ascending=True)


In [14]:
data

,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,User's Transaction History,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag
38034,10000041,3876,487.500,2023-01-10 14:51:07,3551,Venmo,TAI,Auction,Kiosk,180.67.69.36,...,55,3.570312,Egypt,CAD,Rental Payment,739,verizon.co.uk,19,Hardware Token,0
1568542,10000053,4682,886.500,2023-03-06 03:38:47,4349,Cryptocurrency Wallet,RUS,Membership,Server,119.161.47.211,...,62,1.400391,Hong Kong,SEK,Auction Bid,682,yahoo.ca,12,Face ID,1
1187553,10000066,3173,377.500,2022-05-01 01:04:14,4071,Masterpass,UAE,Contribution,Smart Doorbell,164.0.60.235,...,82,3.570312,United Kingdom,MXN,Royalty,636,rediffmail.com,10,Fingerprint,0
4475460,10000083,2113,160.625,2022-06-28 18:01:01,5498,Contactless Payment,SAU,Cashback,Fitness Tracker,123.29.19.29,...,7,1.419922,Italy,EGP,Retail Purchase,321,gmail.co.in,16,Fingerprint,0
2803248,10000086,4931,876.500,2021-07-02 02:39:13,3150,Stripe,ESP,Service Charge,ATM,150.41.148.165,...,48,3.890625,Poland,ILS,Registration Fee,537,zoho.com,6,Push Notification Confirmation,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2632554,99999838,6861,752.500,2022-08-07 01:48:31,9515,Skrill,MAL,Fine,Drone,185.252.6.238,...,36,2.650391,Poland,DKK,Settlement,503,mail.com,11,CAPTCHA,1
5870485,99999840,3858,692.000,2022-03-13 22:41:45,8569,Google Wallet,IDN,Auction,Fitness Tracker,16.233.55.100,...,18,4.648438,Poland,BRL,Product Purchase,783,roadrunner.co.uk,20,Face ID,1
3596052,99999853,4629,149.125,2022-12-02 18:16:10,8842,Stripe,ISR,Withdrawal,Vehicle Infotainment System,176.199.153.153,...,17,2.679688,Malaysia,CHF,Compensation,743,rocketmail.co.uk,15,Radio-Frequency Identification (RFID),0
2926699,99999881,5012,586.000,2021-10-18 10:05:35,8291,Worldpay,ZAF,Payout,Smart Speaker,54.44.199.25,...,11,1.049805,Singapore,PLN,Insurance Premium,714,mail.com,9,Radio-Frequency Identification (RFID),0


In [15]:
df = data[data.duplicated(subset='Transaction ID', keep=False)]

In [16]:
df

,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,User's Transaction History,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag
843526,10000410,3929,817.500,2022-02-02 18:50:03,8303,Debit Card,IND,Cashback,Smart Speaker,103.66.87.245,...,10,2.650391,Thailand,JPY,Interest Payment,636,fastmail.com,13,Transaction Confirmation Number,1
4234056,10000410,9627,830.500,2021-09-04 08:55:33,5483,Money Order,NZL,Service Charge,Smart Mirror,142.41.187.13,...,63,1.740234,Belgium,SEK,Invoice Payment,502,protonmail.com,11,SMS Code,0
2113737,10000417,8842,59.375,2021-12-10 20:17:26,7162,WeChat Pay,MAL,Auction,Server,49.111.133.12,...,79,2.369141,France,SAR,Tax Payment,405,icloud.com,11,SMS Code,0
3182335,10000417,4986,310.500,2022-01-19 04:35:55,4922,Diners Club,BRA,Interest,Embedded System,14.201.154.187,...,75,3.439453,Israel,VND,Transfer to Family,700,zoho.co.uk,19,Pattern Lock,0
5043236,10000431,1714,270.500,2022-02-20 17:12:03,6683,Afterpay,CHE,Investment,Smart Appliance,27.186.38.3,...,13,3.800781,Austria,SAR,Retail Purchase,692,yandex.co.uk,18,SMS Code,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199035,99998589,1746,560.500,2021-11-13 06:35:16,4834,Square Cash,UK,Rent,E-Reader,87.63.155.115,...,87,3.580078,South Korea,NOK,Donation to Nonprofit,734,yahoo.co.uk,7,Handwriting Recognition,1
5196701,99999703,8074,34.875,2021-01-16 19:42:16,3371,Debit Card,CAN,Charity,Robot,13.152.24.35,...,56,3.570312,Argentina,CLP,Auction Bid,764,zoho.co.uk,12,CAPTCHA,1
4935056,99999703,6763,137.875,2021-08-21 12:33:59,8556,E-check,UK,Loan,IoT Device,200.64.78.141,...,32,3.660156,Indonesia,PEN,Ticket Purchase,505,mail.com,7,PIN,1
3769431,99999788,1476,56.125,2022-11-15 07:56:00,3560,Bank Transfer,RUS,Transfer,Desktop,119.20.227.70,...,51,2.750000,Netherlands,NZD,Fine Settlement,541,gmail.co.uk,19,USB Security Key,1


In [17]:
customer_id_counts = data['Transaction ID'].value_counts()

# Find the ID with the highest number of duplicates
max_duplicates_id = customer_id_counts.idxmax()
max_duplicates_count = customer_id_counts.max()

In [18]:
max_duplicates_count

4

In [19]:
data

,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,User's Transaction History,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag
38034,10000041,3876,487.500,2023-01-10 14:51:07,3551,Venmo,TAI,Auction,Kiosk,180.67.69.36,...,55,3.570312,Egypt,CAD,Rental Payment,739,verizon.co.uk,19,Hardware Token,0
1568542,10000053,4682,886.500,2023-03-06 03:38:47,4349,Cryptocurrency Wallet,RUS,Membership,Server,119.161.47.211,...,62,1.400391,Hong Kong,SEK,Auction Bid,682,yahoo.ca,12,Face ID,1
1187553,10000066,3173,377.500,2022-05-01 01:04:14,4071,Masterpass,UAE,Contribution,Smart Doorbell,164.0.60.235,...,82,3.570312,United Kingdom,MXN,Royalty,636,rediffmail.com,10,Fingerprint,0
4475460,10000083,2113,160.625,2022-06-28 18:01:01,5498,Contactless Payment,SAU,Cashback,Fitness Tracker,123.29.19.29,...,7,1.419922,Italy,EGP,Retail Purchase,321,gmail.co.in,16,Fingerprint,0
2803248,10000086,4931,876.500,2021-07-02 02:39:13,3150,Stripe,ESP,Service Charge,ATM,150.41.148.165,...,48,3.890625,Poland,ILS,Registration Fee,537,zoho.com,6,Push Notification Confirmation,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2632554,99999838,6861,752.500,2022-08-07 01:48:31,9515,Skrill,MAL,Fine,Drone,185.252.6.238,...,36,2.650391,Poland,DKK,Settlement,503,mail.com,11,CAPTCHA,1
5870485,99999840,3858,692.000,2022-03-13 22:41:45,8569,Google Wallet,IDN,Auction,Fitness Tracker,16.233.55.100,...,18,4.648438,Poland,BRL,Product Purchase,783,roadrunner.co.uk,20,Face ID,1
3596052,99999853,4629,149.125,2022-12-02 18:16:10,8842,Stripe,ISR,Withdrawal,Vehicle Infotainment System,176.199.153.153,...,17,2.679688,Malaysia,CHF,Compensation,743,rocketmail.co.uk,15,Radio-Frequency Identification (RFID),0
2926699,99999881,5012,586.000,2021-10-18 10:05:35,8291,Worldpay,ZAF,Payout,Smart Speaker,54.44.199.25,...,11,1.049805,Singapore,PLN,Insurance Premium,714,mail.com,9,Radio-Frequency Identification (RFID),0


In [20]:
data['TransactionDuplicate'] = '1Dup'

In [21]:
data

,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag,TransactionDuplicate
38034,10000041,3876,487.500,2023-01-10 14:51:07,3551,Venmo,TAI,Auction,Kiosk,180.67.69.36,...,3.570312,Egypt,CAD,Rental Payment,739,verizon.co.uk,19,Hardware Token,0,1Dup
1568542,10000053,4682,886.500,2023-03-06 03:38:47,4349,Cryptocurrency Wallet,RUS,Membership,Server,119.161.47.211,...,1.400391,Hong Kong,SEK,Auction Bid,682,yahoo.ca,12,Face ID,1,1Dup
1187553,10000066,3173,377.500,2022-05-01 01:04:14,4071,Masterpass,UAE,Contribution,Smart Doorbell,164.0.60.235,...,3.570312,United Kingdom,MXN,Royalty,636,rediffmail.com,10,Fingerprint,0,1Dup
4475460,10000083,2113,160.625,2022-06-28 18:01:01,5498,Contactless Payment,SAU,Cashback,Fitness Tracker,123.29.19.29,...,1.419922,Italy,EGP,Retail Purchase,321,gmail.co.in,16,Fingerprint,0,1Dup
2803248,10000086,4931,876.500,2021-07-02 02:39:13,3150,Stripe,ESP,Service Charge,ATM,150.41.148.165,...,3.890625,Poland,ILS,Registration Fee,537,zoho.com,6,Push Notification Confirmation,0,1Dup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2632554,99999838,6861,752.500,2022-08-07 01:48:31,9515,Skrill,MAL,Fine,Drone,185.252.6.238,...,2.650391,Poland,DKK,Settlement,503,mail.com,11,CAPTCHA,1,1Dup
5870485,99999840,3858,692.000,2022-03-13 22:41:45,8569,Google Wallet,IDN,Auction,Fitness Tracker,16.233.55.100,...,4.648438,Poland,BRL,Product Purchase,783,roadrunner.co.uk,20,Face ID,1,1Dup
3596052,99999853,4629,149.125,2022-12-02 18:16:10,8842,Stripe,ISR,Withdrawal,Vehicle Infotainment System,176.199.153.153,...,2.679688,Malaysia,CHF,Compensation,743,rocketmail.co.uk,15,Radio-Frequency Identification (RFID),0,1Dup
2926699,99999881,5012,586.000,2021-10-18 10:05:35,8291,Worldpay,ZAF,Payout,Smart Speaker,54.44.199.25,...,1.049805,Singapore,PLN,Insurance Premium,714,mail.com,9,Radio-Frequency Identification (RFID),0,1Dup


In [22]:
data = data.sort_values(by='Transaction Date and Time', ascending=True)

In [23]:
# Identify duplicates in columnB and mark occurrences
data['occurrence'] = data.groupby('Transaction ID').cumcount() + 1

# Update columnA based on occurrence
data['TransactionDuplicate']  = data.apply(lambda row: f'{row["occurrence"]}Dup' if row['occurrence'] > 1 else row['TransactionDuplicate'], axis=1)

# Drop the 'occurrence' column as it's no longer needed
data.drop('occurrence', axis=1, inplace=True)

# Find the non-duplicate values in ColumnB
non_duplicates = data['Transaction ID'].value_counts() == 1
non_duplicate_values = non_duplicates[non_duplicates].index

# Replace the values in ColumnA where corresponding value in ColumnB is not a duplicate
data.loc[data['Transaction ID'].isin(non_duplicate_values), 'TransactionDuplicate'] = 'NonDup'

In [24]:
data['TransactionDuplicate'].unique()

array(['NonDup', '1Dup', '2Dup', '3Dup', '4Dup'], dtype=object)

In [25]:
data = data.sort_values(by='Transaction ID', ascending=True)

In [26]:
duplicate_rows = data[data.duplicated('Transaction ID', keep=False)]
print("Duplicate Rows based on Transaction ID:")
duplicate_rows

Duplicate Rows based on Transaction ID:


,Transaction ID,User ID,Transaction Amount,Transaction Date and Time,Merchant ID,Payment Method,Country Code,Transaction Type,Device Type,IP Address,...,Merchant's Reputation Score,User's Device Location,Transaction Currency,Transaction Purpose,User's Credit Score,User's Email Domain,Merchant's Business Age,Transaction Authentication Method,Fraudulent Flag,TransactionDuplicate
843526,10000410,3929,817.500,2022-02-02 18:50:03,8303,Debit Card,IND,Cashback,Smart Speaker,103.66.87.245,...,2.650391,Thailand,JPY,Interest Payment,636,fastmail.com,13,Transaction Confirmation Number,1,2Dup
4234056,10000410,9627,830.500,2021-09-04 08:55:33,5483,Money Order,NZL,Service Charge,Smart Mirror,142.41.187.13,...,1.740234,Belgium,SEK,Invoice Payment,502,protonmail.com,11,SMS Code,0,1Dup
2113737,10000417,8842,59.375,2021-12-10 20:17:26,7162,WeChat Pay,MAL,Auction,Server,49.111.133.12,...,2.369141,France,SAR,Tax Payment,405,icloud.com,11,SMS Code,0,1Dup
3182335,10000417,4986,310.500,2022-01-19 04:35:55,4922,Diners Club,BRA,Interest,Embedded System,14.201.154.187,...,3.439453,Israel,VND,Transfer to Family,700,zoho.co.uk,19,Pattern Lock,0,2Dup
5043236,10000431,1714,270.500,2022-02-20 17:12:03,6683,Afterpay,CHE,Investment,Smart Appliance,27.186.38.3,...,3.800781,Austria,SAR,Retail Purchase,692,yandex.co.uk,18,SMS Code,1,2Dup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472682,99998589,4271,727.500,2021-04-19 16:42:45,3965,Cash,RUS,Purchase,Cash Register,149.227.135.128,...,2.869141,Italy,BRL,Buyback,787,yahoo.com,18,Geolocation Verification,0,1Dup
4935056,99999703,6763,137.875,2021-08-21 12:33:59,8556,E-check,UK,Loan,IoT Device,200.64.78.141,...,3.660156,Indonesia,PEN,Ticket Purchase,505,mail.com,7,PIN,1,2Dup
5196701,99999703,8074,34.875,2021-01-16 19:42:16,3371,Debit Card,CAN,Charity,Robot,13.152.24.35,...,3.570312,Argentina,CLP,Auction Bid,764,zoho.co.uk,12,CAPTCHA,1,1Dup
3769431,99999788,1476,56.125,2022-11-15 07:56:00,3560,Bank Transfer,RUS,Transfer,Desktop,119.20.227.70,...,2.750000,Netherlands,NZD,Fine Settlement,541,gmail.co.uk,19,USB Security Key,1,2Dup


In [27]:
data['Transaction Date and Time'] = pd.to_datetime(data['Transaction Date and Time'])

# Features Engineering

#### Bining the User Age in groups of 10

In [28]:
data['User Age'].unique()

array([54, 33, 76, 39, 44, 55, 31, 50, 68, 48, 79, 28, 65, 62, 45, 58, 57,
       18, 53, 41, 38, 35, 42, 56, 34, 49, 25, 36, 66, 24, 69, 21, 70, 52,
       20, 29, 61, 71, 26, 30, 47, 32, 19, 46, 43, 63, 67, 74, 60, 73, 22,
       72, 23, 37, 64, 40, 78, 75, 80, 51, 59, 27, 77], dtype=int8)

In [29]:

ages = [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 
        38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 
        58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 
        78, 79, 80]

# Define bin edges
bin_edges = list(range(10, 100, 10))

# Create labels for bins
bin_labels = [f'Age_Group {i+1}' for i in range(len(bin_edges) - 1)]

# Bin the 'User Age' column using the sorted bin edges and labels
data['age_group'] = pd.cut(data['User Age'], bins=bin_edges, labels=bin_labels, right=False)

# Print the binned DataFrame
print(data[['User Age', 'age_group']])

         User Age    age_group
38034          54  Age_Group 5
1568542        33  Age_Group 3
1187553        76  Age_Group 7
4475460        39  Age_Group 3
2803248        44  Age_Group 4
...           ...          ...
2632554        35  Age_Group 3
5870485        62  Age_Group 6
3596052        70  Age_Group 7
2926699        57  Age_Group 5
4860277        22  Age_Group 2

[6000000 rows x 2 columns]


#### Bining the Business Age

In [30]:
bage_sort = data["Merchant's Business Age"].unique()

In [31]:
bage_sort
biz_age = np.sort(bage_sort)

In [32]:

# Define bin edges
bin_edges = list(range(0, 30, 5))

# Create labels for bins
bin_labels = [f'Group_biz {i+1}' for i in range(len(bin_edges) - 1)]

# Bin the 'User Age' column using the sorted bin edges and labels
data['biz_age_group'] = pd.cut(data["Merchant's Business Age"], bins=bin_edges, labels=bin_labels, right=False)

# Print the binned DataFrame
print(data[["Merchant's Business Age", 'biz_age_group']])

         Merchant's Business Age biz_age_group
38034                         19   Group_biz 4
1568542                       12   Group_biz 3
1187553                       10   Group_biz 3
4475460                       16   Group_biz 4
2803248                        6   Group_biz 2
...                          ...           ...
2632554                       11   Group_biz 3
5870485                       20   Group_biz 5
3596052                       15   Group_biz 4
2926699                        9   Group_biz 2
4860277                        8   Group_biz 2

[6000000 rows x 2 columns]


#### Bining the Merchant's Reputation Score

In [33]:
merch = data["Merchant's Reputation Score"].unique()

In [34]:
merch_score = np.sort(merch)
merch_score

array([1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1 ,
       1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 , 1.21,
       1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31, 1.32,
       1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42, 1.43,
       1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.5 , 1.51, 1.52, 1.53, 1.54,
       1.55, 1.56, 1.57, 1.58, 1.59, 1.6 , 1.61, 1.62, 1.63, 1.64, 1.65,
       1.66, 1.67, 1.68, 1.69, 1.7 , 1.71, 1.72, 1.73, 1.74, 1.75, 1.76,
       1.77, 1.78, 1.79, 1.8 , 1.81, 1.82, 1.83, 1.84, 1.85, 1.86, 1.87,
       1.88, 1.89, 1.9 , 1.91, 1.92, 1.93, 1.94, 1.95, 1.96, 1.97, 1.98,
       1.99, 2.  , 2.01, 2.02, 2.03, 2.04, 2.05, 2.06, 2.07, 2.08, 2.09,
       2.1 , 2.11, 2.12, 2.13, 2.14, 2.15, 2.16, 2.17, 2.18, 2.19, 2.2 ,
       2.21, 2.22, 2.23, 2.24, 2.25, 2.26, 2.27, 2.28, 2.29, 2.3 , 2.31,
       2.32, 2.33, 2.34, 2.35, 2.36, 2.37, 2.38, 2.39, 2.4 , 2.41, 2.42,
       2.43, 2.44, 2.45, 2.46, 2.47, 2.48, 2.49, 2.

In [35]:
# Define bin edges
bin_edges = [i / 2 for i in range(2, 50)]

# Create labels for bins
bin_labels = [f'reputation_group {i+1}' for i in range(len(bin_edges) - 1)]

# Bin the 'User Age' column using the sorted bin edges and labels
data['merch_rep_score'] = pd.cut(data["Merchant's Reputation Score"], bins=bin_edges, labels=bin_labels, right=False)

# Print the binned DataFrame
print(data[["Merchant's Reputation Score", 'merch_rep_score']])

         Merchant's Reputation Score     merch_rep_score
38034                       3.570312  reputation_group 6
1568542                     1.400391  reputation_group 1
1187553                     3.570312  reputation_group 6
4475460                     1.419922  reputation_group 1
2803248                     3.890625  reputation_group 6
...                              ...                 ...
2632554                     2.650391  reputation_group 4
5870485                     4.648438  reputation_group 8
3596052                     2.679688  reputation_group 4
2926699                     1.049805  reputation_group 1
4860277                     1.160156  reputation_group 1

[6000000 rows x 2 columns]


#### Bining the User Credit Score

In [36]:
cscore = data["User's Credit Score"].unique()

In [37]:
credit_score = np.sort(cscore)
credit_score

array([300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312,
       313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325,
       326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338,
       339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351,
       352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364,
       365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390,
       391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403,
       404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416,
       417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429,
       430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442,
       443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455,
       456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468,
       469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 48

In [38]:
# Define the bins and labels
bins = [300, 550, 650, 750, 900]
labels = ['Poor', 'Average', 'Good', 'Excellent']

# Use cut to categorize the data
data['Credit_Category'] = pd.cut(data["User's Credit Score"], bins=bins, labels=labels, right=False)

In [39]:
print(data[["User's Credit Score", 'Credit_Category']])

         User's Credit Score Credit_Category
38034                    739            Good
1568542                  682            Good
1187553                  636         Average
4475460                  321            Poor
2803248                  537            Poor
...                      ...             ...
2632554                  503            Poor
5870485                  783       Excellent
3596052                  743            Good
2926699                  714            Good
4860277                  480            Poor

[6000000 rows x 2 columns]


# Exploratory Data Analysis (EDA)

#### 1. Number of Transactions by Country
#### The Countries with their number of transactions are arranged in descending order of Volume.

In [40]:
data['Country Code'].value_counts()

Country Code
HKG    150917
UK     150840
IDN    150776
VNM    150545
BEL    150533
GER    150518
ARG    150496
SAU    150456
AUT    150452
CHN    150263
MAL    150249
RUS    150164
NGA    150140
FRA    150139
POL    150128
SGP    150127
MEX    150113
TAI    150080
ISR    150069
JPN    150059
BRA    150050
AUS    150041
USA    150031
THA    149965
IND    149896
QAT    149838
NLD    149829
KOR    149805
UAE    149735
CHE    149710
SWE    149572
NZL    149558
CAN    149506
EGY    149471
NOR    149463
ZAF    149439
ITA    149338
TUR    149333
ESP    149309
GRE    149047
Name: count, dtype: int64

#### 2. Total Number of Users on the Platform

In [41]:
users = data['User ID'].nunique()
print("The total number of users on the platform:", users, "users")

The total number of users on the platform: 9000 users


#### 3. Average Monthly Users yearly from 2021 - 2023

In [42]:
# Extract year and month from the datetime column
data['year'] = data['Transaction Date and Time'].dt.year
data['month'] = data['Transaction Date and Time'].dt.month

# Filter data for the years 2021 to 2023
df_filtered = data[(data['year'] >= 2021) & (data['year'] <= 2023)]

# Group by year and month, then count the unique users for each month
monthly_user_counts = df_filtered.groupby(['year', 'month'])['User ID'].nunique()

# Calculate the average number of monthly users for each year
average_monthly_users_per_year = monthly_user_counts.groupby('year').mean()

print(average_monthly_users_per_year)

year
2021    9000.0
2022    9000.0
2023    9000.0
Name: User ID, dtype: float64


#### 4. Average Monthly Transactions yearly from 2021 - 2023

In [43]:
# Group by year and month, then count the unique transactions for each month
monthly_transactions_counts = df_filtered.groupby(['year', 'month'])['Transaction ID'].nunique()

# Calculate the average number of monthly transactions for each year
average_monthly_transactions_per_year = monthly_transactions_counts.groupby('year').mean()

print(average_monthly_transactions_per_year)

year
2021    193828.833333
2022    193761.750000
2023    191796.000000
Name: Transaction ID, dtype: float64


# Data Encoding

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6000000 entries, 38034 to 4860277
Data columns (total 39 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   Transaction ID                     int32         
 1   User ID                            int16         
 2   Transaction Amount                 float16       
 3   Transaction Date and Time          datetime64[ns]
 4   Merchant ID                        int16         
 5   Payment Method                     category      
 6   Country Code                       category      
 7   Transaction Type                   category      
 8   Device Type                        category      
 9   IP Address                         category      
 10  Browser Type                       category      
 11  Operating System                   category      
 12  Merchant Category                  category      
 13  User Age                           int8          
 14  Use

#### One-Hot Encoding

In [45]:
#columns_to_encode = ['age_group', 'biz_age_group', 'merch_rep_score', 'Credit_Category', 'TransactionDuplicate']

# Perform one-hot encoding
#df_encoded = pd.get_dummies(data[columns_to_encode], prefix=columns_to_encode)

# Concatenate the one-hot encoded columns with the original DataFrame
#df_final = pd.concat([data.drop(columns_to_encode, axis=1), df_encoded], axis=1)

In [47]:
#df_final

In [48]:
# Replace 'true' with 1 and 'false' with 0
#df_final.replace({True: 1, False: 0}, inplace=True)

In [49]:
dataset = ['Transaction Date and Time', 'IP Address', 'Transaction ID', 'User ID', 'Merchant ID', "User's Email Domain"]
data.drop(dataset, axis = 1, inplace = True)

#### Label Encoding

In [50]:
categorical_columns = df.select_dtypes(include=['category']).columns

# Convert categorical columns to an array
categorical_columns_array = np.array(categorical_columns)

# Print the array of categorical columns
print("Categorical Columns as an Array:")
print(categorical_columns_array)

Categorical Columns as an Array:
['Transaction Date and Time' 'Payment Method' 'Country Code'
 'Transaction Type' 'Device Type' 'IP Address' 'Browser Type'
 'Operating System' 'Merchant Category' 'User Occupation' 'User Gender'
 'User Account Status' 'Transaction Status' 'Transaction Time of Day'
 "User's Device Location" 'Transaction Currency' 'Transaction Purpose'
 "User's Email Domain" 'Transaction Authentication Method']


In [55]:
# For Python with scikit-learn
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

categorical_columns = ['Payment Method', 'Country Code',
 'Transaction Type', 'Device Type', 'Browser Type',
 'Operating System', 'Merchant Category', 'User Occupation', 'User Gender',
 'User Account Status', 'Transaction Status', 'Transaction Time of Day',
 "User's Device Location", 'Transaction Currency', 'Transaction Purpose', 
 'Transaction Authentication Method', 'age_group', 'biz_age_group', 'merch_rep_score', 'Credit_Category', 'TransactionDuplicate']

for col in categorical_columns:
    data[col + '_encoded'] = label_encoder.fit_transform(data[col])

data.drop(categorical_columns, axis=1, inplace=True)  


KeyError: 'Payment Method'

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6000000 entries, 38034 to 4860277
Data columns (total 33 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   Transaction Amount                         float16
 1   User Age                                   int8   
 2   User Income                                float32
 3   Location Distance                          float16
 4   Time Taken for Transaction                 float16
 5   User's Transaction History                 int8   
 6   Merchant's Reputation Score                float16
 7   User's Credit Score                        int16  
 8   Merchant's Business Age                    int8   
 9   Fraudulent Flag                            int8   
 10  year                                       int32  
 11  month                                      int32  
 12  Payment Method_encoded                     int64  
 13  Country Code_encoded                       

# Modeling

In [58]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = data.drop('Fraudulent Flag', axis=1)
y = data['Fraudulent Flag']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Set hyperparameters for the LightGBM model
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'device_type': 'gpu'
}

# Train the LightGBM model
num_round = 1000
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data], early_stopping_rounds=50, verbose_eval=100)

# Predict on the test set
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred_binary = np.round(y_pred)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_binary)
print('Accuracy:', accuracy)

report = classification_report(y_test, y_pred_binary)
print('Classification Report:\n', report)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2100034, number of negative: 2099966
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2282
[LightGBM] [Info] Number of data points in the train set: 4200000, number of used features: 32
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 31 dense feature groups (128.17 MB) transferred to GPU in 0.193073 secs. 1 sparse feature groups
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500008 -> initscore=0.000032
[LightGBM] [Info] Start training from score 0.000032
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[11]	valid_0's binary